In [1]:
import json
import nltk
import tensorflow
import random
import numpy as np

In [2]:
intents= json.loads(open(r"../datasets/chatbot/intents.json").read())

In [3]:
lemmatizer= nltk.stem.WordNetLemmatizer()

In [4]:
words = [] 
classes = [] 
documents_x = [] 
documents_y= []
ignore_letters = ["?", "!", ".", ","] 

In [5]:
for intent in intents['intents']: 
    for pattern in intent['patterns']: 
        word_list = nltk.word_tokenize(pattern) 
        words.extend(word_list)

        documents_x.append(word_list)
        documents_y.append(intent['tag'])

        if intent['tag'] not in classes: 
                    classes.append(intent['tag'])

In [6]:
nltk.download('wordnet')
words = [lemmatizer.lemmatize(word.lower()) 
         for word in words if word not in ignore_letters] 
words = sorted(list(set(words))) 
classes= sorted(list(classes))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rohan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
import pickle
pickle.dump(words, open('words.pkl', 'wb')) 
pickle.dump(classes, open('classes.pkl', 'wb')) 

In [8]:
training= []
output= []
output_empty = [0 for _ in range(len(classes))]
for x, document in enumerate(documents_x): 
    bag = [] 
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in document]

    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = output_empty[:]
    output_row[classes.index(documents_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [9]:
training = np.array(training)
output= np.array(output)

In [10]:
model= tensorflow.keras.models.Sequential()
model.add(tensorflow.keras.layers.Dense(128, input_shape=(len(training[0]),), activation='relu'))
model.add(tensorflow.keras.layers.Dropout(0.5))
model.add(tensorflow.keras.layers.Dense(64, activation='relu'))
model.add(tensorflow.keras.layers.Dropout(0.5))
model.add(tensorflow.keras.layers.Dense(len(output[0]), activation='softmax'))

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(training, output, epochs=200, batch_size=5)

Epoch 1/200
4/4 [==============================] - 2s 7ms/step - loss: 1.3922 - accuracy: 0.2500
Epoch 2/200
4/4 [==============================] - 0s 7ms/step - loss: 1.4027 - accuracy: 0.2500
Epoch 3/200
4/4 [==============================] - 0s 6ms/step - loss: 1.3782 - accuracy: 0.1500
Epoch 4/200
4/4 [==============================] - 0s 5ms/step - loss: 1.3415 - accuracy: 0.3000
Epoch 5/200
4/4 [==============================] - 0s 7ms/step - loss: 1.3626 - accuracy: 0.2500
Epoch 6/200
4/4 [==============================] - 0s 6ms/step - loss: 1.3009 - accuracy: 0.3500
Epoch 7/200
4/4 [==============================] - 0s 7ms/step - loss: 1.3674 - accuracy: 0.2500
Epoch 8/200
4/4 [==============================] - 0s 7ms/step - loss: 1.1754 - accuracy: 0.5000
Epoch 9/200
4/4 [==============================] - 0s 7ms/step - loss: 1.2102 - accuracy: 0.5500
Epoch 10/200
4/4 [==============================] - 0s 7ms/step - loss: 1.1661 - accuracy: 0.5500
Epoch 11/200
4/4 [===========

In [12]:
model.save('chatbot_model.h5')

In [13]:
with open('data.pickle', 'wb') as f:
    pickle.dump((words, classes, training, output), f)

In [14]:
def bag_of_words(s, words): 
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [lemmatizer.lemmatize(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    return np.array(bag)

In [39]:
def chat(prompt: str):

    results = model.predict(np.array([bag_of_words(prompt, words)]))[0]
    results_index = np.argmax(results)
    tag = classes[results_index]

    for tg in intents['intents']:
        if tg['tag'] == tag:
            responses = tg['responses']
    print(random.choice(responses))